In [ ]:
!pip install scikit-learn

In [16]:
import os
import pandas as pd

# Função para ler o arquivo metadata.txt e obter o nome do artista e o gênero musical
def ler_metadata(metadata_file):
    with open(metadata_file, 'r', encoding='utf-8') as f:
        artist_name = f.readline().strip()
        genre = f.readline().strip()
    return artist_name, genre

# Função para ler o conteúdo do arquivo .txt (letra da música)
def ler_letra(letra_file):
    with open(letra_file, 'r', encoding='utf-8') as f:
        letra = f.read()
    return letra

# Função para ler o conteúdo do arquivo .html (cifra da música)
def ler_cifra(cifra_file):
    with open(cifra_file, 'r', encoding='utf-8') as f:
        cifra = f.read()
    return cifra

# Lista para armazenar os dados de cada música
data = []
# Percorrer recursivamente as pastas de artistas
for root, dirs, files in os.walk('/content/'):

    if '.config' in dirs:
        dirs.remove('.config')
    if '.ipynb_checkpoints' in dirs:
        dirs.remove('.ipynb_checkpoints')
    if 'sample_data' in dirs:
        dirs.remove('sample_data')

    artist_name, genre, letra, cifra = ["", "", "", ""]

    if os.path.exists(os.path.join(root,'metadata.txt')):
        artist_name, genre = ler_metadata(os.path.join(root,'metadata.txt'))

    for file in files:
        m_file = os.path.join(root, file)

        if '.html' in file:
            cifra = ler_cifra(m_file)
        else:
            letra = ler_letra(m_file)

        data.append({'Artist': artist_name, 'Genre': genre, 'Lyrics': letra, 'Chords': cifra})

# Criar um DataFrame com os dados coletados
df = pd.DataFrame(data)

# Visualizar o DataFrame
print(df.head())


metadata.txt
Amazing_cifra.html
AindaCedo.txt
MetalContraasNuvens.txt
EuSei.txt
metadata.txt
Always.txt
LivinOnaPrayer.txt
BedOfRoses.txt
BlazeOfGlory.txt
Legendary_cifra.html
          Artist     Genre                                             Lyrics  \
0  Legião Urbana  Pop Rock                            Legião Urbana\nPop Rock   
1  Legião Urbana  Pop Rock                            Legião Urbana\nPop Rock   
2  Legião Urbana  Pop Rock  Uma menina me ensinou\nQuase tudo que eu sei\n...   
3  Legião Urbana  Pop Rock  Não sou escravo de ninguém\nNinguémsenhor do m...   
4  Legião Urbana  Pop Rock  Sexo verbal não faz meu estilo\nPalavras são e...   

                                              Chords  
0                                                     
1  <html><h1>Amazing<h1><div><pre>Intro: <b>Am</b...  
2  <html><h1>Amazing<h1><div><pre>Intro: <b>Am</b...  
3  <html><h1>Amazing<h1><div><pre>Intro: <b>Am</b...  
4  <html><h1>Amazing<h1><div><pre>Intro: <b>Am</b...  


In [24]:
popularidade_generos = df['Genre'].value_counts()

# Visualizar os resultados
print(popularidade_generos)

Genre
Hard Rock    6
Pop Rock     5
Name: count, dtype: int64


In [32]:
df.groupby(['Artist', 'Genre']).size()

Artist         Genre    
Bon Jovi       Hard Rock    6
Legião Urbana  Pop Rock     5
dtype: int64

In [35]:
df['comprimento_letra'] = df['Lyrics'].str.len()
df

,Artist,Genre,Lyrics,Chords,comprimento_letra
0,Legião Urbana,Pop Rock,Legião Urbana\nPop Rock,,22
1,Legião Urbana,Pop Rock,Legião Urbana\nPop Rock,<html><h1>Amazing<h1><div><pre>Intro: <b>Am</b...,22
2,Legião Urbana,Pop Rock,Uma menina me ensinou\nQuase tudo que eu sei\n...,<html><h1>Amazing<h1><div><pre>Intro: <b>Am</b...,979
3,Legião Urbana,Pop Rock,Não sou escravo de ninguém\nNinguémsenhor do m...,<html><h1>Amazing<h1><div><pre>Intro: <b>Am</b...,1715
4,Legião Urbana,Pop Rock,Sexo verbal não faz meu estilo\nPalavras são e...,<html><h1>Amazing<h1><div><pre>Intro: <b>Am</b...,507
5,Bon Jovi,Hard Rock,Bon Jovi\nHard Rock,,18
6,Bon Jovi,Hard Rock,Always\n \nThis Romeo is bleeding \nBut you ca...,,1878
7,Bon Jovi,Hard Rock,"Livin' On a Prayer\n \n(Once upon a time, not ...",,1643
8,Bon Jovi,Hard Rock,"Bed Of Roses\n \nSitting here, wasted and woun...",,2151
9,Bon Jovi,Hard Rock,Blaze Of Glory\n \nI wake up in the morning an...,,1884


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

# Criar uma matriz de frequência de palavras
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Lyrics'])

# Obter as palavras mais comuns
palavras_comuns = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out()).sum().sort_values(ascending=False)

# Visualizar as palavras mais comuns
print(palavras_comuns.head(10))

of       61
you      58
the      50
to       45
me       43
and      41
on       40
in       38
glory    30
blaze    30
dtype: int64


In [29]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [31]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.sentiment.vader import SentimentIntensityAnalyzer


# Função para ler todos os arquivos de texto em um diretório
def ler_arquivos_texto(diretorio):
    textos = []
    for arquivo in os.listdir(diretorio):
        print(arquivo)
        if arquivo.endswith(".txt"):
            with open(os.path.join(diretorio, arquivo), "r", encoding="utf-8") as f:
                sia = SentimentIntensityAnalyzer()
                t = f.read()
                sentiment_score = sia.polarity_scores(t)
                print(sentiment_score)
                textos.append(t)

    return textos

# Diretório onde os arquivos de texto estão localizados
diretorio = "/content/bonjovi/"

# Ler os arquivos de texto
textos = ler_arquivos_texto(diretorio)

# Vetorização dos textos usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(textos)

# Execução do algoritmo KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

# Exibir os clusters e suas palavras-chave
print("Palavras-chave dos clusters:")
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(3):  # Altere o número de clusters conforme necessário
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:  # Altere o número de palavras-chave exibidas conforme necessário
        print(' %s' % terms[ind])


metadata.txt
{'neg': 0.318, 'neu': 0.682, 'pos': 0.0, 'compound': -0.1027}
Always.txt
{'neg': 0.162, 'neu': 0.759, 'pos': 0.078, 'compound': -0.9919}
LivinOnaPrayer.txt
{'neg': 0.057, 'neu': 0.814, 'pos': 0.129, 'compound': 0.986}
BedOfRoses.txt
{'neg': 0.067, 'neu': 0.863, 'pos': 0.07, 'compound': -0.3961}
BlazeOfGlory.txt
{'neg': 0.102, 'neu': 0.617, 'pos': 0.281, 'compound': 0.998}
Legendary_cifra.html
Palavras-chave dos clusters:
Cluster 0:
 prayer
 oh
 livin
 living
 got
 make
 halfway
 hand
 whoa
 ve
Cluster 1:
 bed
 roses
 blaze
 glory
 ll
 just
 words
 love
 till
 ve
Cluster 2:
 bon
 rock
 jovi
 hard
 young
 goin
 given
 giving
 glory
 god
